In [3]:
"""
提取肌力测试中FMG信号特征，用于与超声测量的绝对值归一化结果进行对比

特征包括：
    * 活动态、静息态的绝对值
    * 初始压力
    * 受试者姓名、BMI、年龄
"""
import pandas as pd

from utils_iFEMG_feature import df_save_csv
from utils_gui_model import FMG_overview, FMG_overview_df, form_sbj_info_df

In [40]:
"""
构建空的dataframe，用于填充特征

1. 传单器PCB通道号更改时更改dic

"""
feature_name_list = ['ave', 'std', 'initial_pressure_min', 'initial_pressure_ave', 'act_ave_list', 'act_std_list', 'rst_ave_list', 'rst_std_list']
sbj_info_name_list = ['subject_name', 'height', 'weight', 'age', 'label']

ch1_str = "bicps_br"
ch2_str = "tricps_br_medial"
ch3_str = "tricps_br_lateral"

ch1_name_list = [ch1_str for i in feature_name_list]
ch2_name_list = [ch2_str for i in feature_name_list]
ch3_name_list = [ch3_str for i in feature_name_list]

sbj_info_col1_list = ['subject_info' for i in sbj_info_name_list]

feature_df = pd.DataFrame(columns=pd.MultiIndex.from_product([[ch1_str, ch2_str, ch3_str], feature_name_list]))
sbj_info_df = pd.DataFrame(columns=pd.MultiIndex.from_product([['subject_info'], sbj_info_name_list]))
all_feature_df = pd.concat([sbj_info_df, feature_df], axis = 1)
print(all_feature_df)
print("Init dataframe shape: ", all_feature_df.shape)

Empty DataFrame
Columns: [(subject_info, subject_name), (subject_info, height), (subject_info, weight), (subject_info, age), (subject_info, label), (bicps_br, ave), (bicps_br, std), (bicps_br, initial_pressure_min), (bicps_br, initial_pressure_ave), (bicps_br, act_ave_list), (bicps_br, act_std_list), (bicps_br, rst_ave_list), (bicps_br, rst_std_list), (tricps_br_medial, ave), (tricps_br_medial, std), (tricps_br_medial, initial_pressure_min), (tricps_br_medial, initial_pressure_ave), (tricps_br_medial, act_ave_list), (tricps_br_medial, act_std_list), (tricps_br_medial, rst_ave_list), (tricps_br_medial, rst_std_list), (tricps_br_lateral, ave), (tricps_br_lateral, std), (tricps_br_lateral, initial_pressure_min), (tricps_br_lateral, initial_pressure_ave), (tricps_br_lateral, act_ave_list), (tricps_br_lateral, act_std_list), (tricps_br_lateral, rst_ave_list), (tricps_br_lateral, rst_std_list)]
Index: []

[0 rows x 29 columns]
Init dataframe shape:  (0, 29)


In [41]:
"""
read data
NEED TO CHANGE BEFORE RUNNING:
    1. file_name_list1 文件名
    2. label_list 数据的重量标签
    3. subject_arg_input 受试者的其他特征参数
    4. signal_channel_list 有用信号所在的通道号, ch_name_list 主动肌或拮抗肌, channel_name = "bicps_br"肌肉名称对应。三者需对应
    5. abandon_ms = 1000, 抛弃前一秒数据，尽量确保数据是等长收缩状态
"""

# 存储文件名对应重量标签的dic
bicps_dic = {'bi-0': 0, 'bi-05': 0.5, 'bi-1': 1, 'bi-2': 2}
tricps_dic = {'tri-0': 0, 'tri-05': 0.5, 'tri-1': 1}

# 需要更改的一块
# signal_channel, 信号对应的传感器通道
signal_channel_dic = {"bicps_br": 1,
                      "tricps_br_lateral": 2,
                      "tricps_br_medial": 4}

file_name_dic = tricps_dic
folder_path = r"E:\Data\202403补实验\zx"
subject_args_dict = {'subject_name': "zx",
                     'height': 178,
                     'weight': 70,
                     'age': 24}

# ==============================================================================================================================

# key是文件名，value是对应的重量标签
for key, value in file_name_dic.items():
    # ch1: bicps_br
    temp_fea_df1 = FMG_overview_df(db_file_path = folder_path+'\\' + key + '.db',
                                    time_file_path = folder_path + '\\' + key + '.txt',
                                    signal_channel = signal_channel_dic["bicps_br"],
                                    abandon_ms = 1000,
                                    signal_sample_freq = 1626)
    temp_fea_df1.columns = [ch1_name_list, feature_name_list]
    # ch2: tricps_med
    temp_fea_df2 = FMG_overview_df(db_file_path = folder_path+'\\' + key + '.db',
                                    time_file_path = folder_path + '\\' + key + '.txt',
                                    signal_channel = signal_channel_dic["tricps_br_medial"],
                                    abandon_ms = 1000,
                                    signal_sample_freq = 1626)
    temp_fea_df2.columns = [ch2_name_list, feature_name_list]
    # ch3: tricps_lat
    temp_fea_df3 = FMG_overview_df(db_file_path = folder_path+'\\' + key + '.db',
                                    time_file_path = folder_path + '\\' + key + '.txt',
                                    signal_channel = signal_channel_dic["tricps_br_lateral"],
                                    abandon_ms = 1000,
                                    signal_sample_freq = 1626)
    temp_fea_df3.columns = [ch3_name_list, feature_name_list]

    # 受试者个人信息和重量label
    temp_sbjinfo_df = form_sbj_info_df(df_len = len(temp_fea_df1), **subject_args_dict, label = value)
    temp_sbjinfo_df.columns = [sbj_info_col1_list, sbj_info_name_list]
    
    # 合并df
    temp_df = pd.concat([temp_sbjinfo_df, temp_fea_df1, temp_fea_df2, temp_fea_df3], axis=1)
    print(temp_df.shape)
    all_feature_df = pd.concat([all_feature_df, temp_df], axis = 0, ignore_index=True)
    print(all_feature_df.shape)
    pass
    
all_feature_df.head


(6, 29)
(6, 29)


C:\Users\WeimyMark\AppData\Local\Temp\ipykernel_14184\381154392.py:59: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_feature_df = pd.concat([all_feature_df, temp_df], axis = 0, ignore_index=True)


(6, 29)
(12, 29)
(6, 29)
(18, 29)


<bound method NDFrame.head of    subject_info                             bicps_br             \
   subject_name height weight age label          ave        std   
0            zx    178     70  24     0  1496.347010  36.657255   
1            zx    178     70  24     0  1496.347010  36.657255   
2            zx    178     70  24     0  1496.347010  36.657255   
3            zx    178     70  24     0  1496.347010  36.657255   
4            zx    178     70  24     0  1496.347010  36.657255   
5            zx    178     70  24     0  1496.347010  36.657255   
6            zx    178     70  24   0.5  1681.446461  30.543486   
7            zx    178     70  24   0.5  1681.446461  30.543486   
8            zx    178     70  24   0.5  1681.446461  30.543486   
9            zx    178     70  24   0.5  1681.446461  30.543486   
10           zx    178     70  24   0.5  1681.446461  30.543486   
11           zx    178     70  24   0.5  1681.446461  30.543486   
12           zx    178     70  2

In [42]:
"""
把数据存储进.csv
"""
# 修改文件后缀: 肱二头肌 or 肱三头肌
filename = folder_path + '\\' + subject_args_dict['subject_name'] + "_FMG_tricps_br.csv"

df_save_csv(all_feature_df, filename)

File E:\Data\202403补实验\zx\zx_FMG_tricps_br.csv saved!
